In [43]:
import numpy as np
import pandas as pd

In [44]:
df = pd.read_csv(r"D:\Python\Kaggle_Datasets\AmazonReview_Analysis\Datasets\amazon_reviews.csv")

In [45]:
df.head()

,Text,label
0,This is the best apps acording to a bunch of ...,1
1,This is a pretty good version of the game for ...,1
2,this is a really . there are a bunch of levels...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [46]:
df.isnull().sum()

Text     0
label    0
dtype: int64

In [47]:
df.duplicated().value_counts()

False    19996
Name: count, dtype: int64

In [48]:
df.shape

(19996, 2)

In [49]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Download required resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhati\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhati\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stops = set(stopwords.words('english'))
stops.remove('not')  # Retain negations for sentiment analysis
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text.lower())  # Remove special characters & lowercase
    words = word_tokenize(text)  # Tokenization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stops]  # Stopword removal & lemmatization
    return " ".join(words)

df["Processed_Text"] = df["Text"].apply(preprocess_text)
df.head()


,Text,label,Processed_Text
0,This is the best apps acording to a bunch of ...,1,best apps acording bunch people agree bomb egg...
1,This is a pretty good version of the game for ...,1,pretty good version game free lot different le...
2,this is a really . there are a bunch of levels...,1,really bunch level find golden egg super fun
3,"This is a silly game and can be frustrating, b...",1,silly game frustrating lot fun definitely reco...
4,This is a terrific game on any pad. Hrs of fun...,1,terrific game pad hr fun grandkids love great ...


In [ ]:
X = df["Processed_Text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [52]:
# df['Text'] = df['Text'].apply(word_tokenize)

In [53]:
# df['Text'] = df['Text'].apply(lambda x:" ".join(x))

In [54]:
# X = df['Text']
# y = df['label']

In [55]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
# X_train =tfidf.fit_transform(X_train).toarray()
# X_test = tfidf.transform(X_test).toarray()

In [ ]:
#TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=6000, ngram_range=(1,2))  # Using unigrams & bigrams
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()


In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve 

In [ ]:
# Train the Random Forest Model
rfc = RandomForestClassifier(n_estimators=200, random_state=42)  # Increased trees for better performance
rfc.fit(X_train_tfidf, y_train)
# Model Evaluation
y_pred = rfc.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred)) 


Model Accuracy: 0.8868
              precision    recall  f1-score   support

           0       0.86      0.64      0.73      1220
           1       0.89      0.97      0.93      3779

    accuracy                           0.89      4999
   macro avg       0.88      0.80      0.83      4999
weighted avg       0.89      0.89      0.88      4999



In [60]:
import pickle
import os

# Ensure "models" directory exists
os.makedirs("models", exist_ok=True)

# Save model and vectorizer
with open(r"..\models\sentiment_model.pkl", "wb") as model_file:
    pickle.dump(rfc, model_file)

with open(r"..\models\tfidf_vectorizer.pkl", "wb") as vectorizer_file:
    pickle.dump(tfidf, vectorizer_file)

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!
